# Tutorial 4: Algorithm Library — Post-Processing & Optimization

This tutorial covers the **le_algorithm** Python module, which provides
post-processing, accuracy evaluation, parameter profiling, and automated
hyperparameter search for line segment detection pipelines.

## What You Will Learn

1. **LineMerge** — merge collinear or near-collinear segments
2. **LineConnect** — connect nearby segments via gradient evidence
3. **AccuracyMeasure** — compute precision, recall, F1, and structural AP
4. **GroundTruthLoader** — load and save ground truth annotations
5. **ImageAnalyzer** — extract image properties (contrast, noise, edges, range)
6. **DetectorProfile** — translate 4 intuitive knobs into detector parameters
7. **ParamOptimizer** — automated parameter search (grid + random)

## Prerequisites

- Completed **Tutorial 1: Library Fundamentals** (or equivalent knowledge)
- Built the LE Python bindings: `bazel build //libs/...`
- Python kernel set to the project `.venv`

## 1. Setup & Imports

In [ ]:
import sys, pathlib

# --- Locate workspace root and add Bazel output dirs to sys.path ---
workspace = pathlib.Path.cwd()
while not (workspace / "MODULE.bazel").exists():
    if workspace == workspace.parent:
        raise RuntimeError("Cannot find LineExtraction workspace root (MODULE.bazel)")
    workspace = workspace.parent

for lib in ["imgproc", "edge", "geometry", "eval", "lsd", "algorithm"]:
    p = workspace / f"bazel-bin/libs/{lib}/python"
    if p.exists():
        sys.path.insert(0, str(p))
    else:
        print(f"Warning: Not found: {p}  — run: bazel build //libs/{lib}/...")

sys.path.insert(0, str(workspace / "python"))

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# NOTE: Do NOT import cv2 — the LE bindings ship their own statically linked
# OpenCV build. Using pip's cv2 would cause symbol conflicts.

import le_algorithm as alg
import le_geometry as geo

print("Imports OK")

### Helper Functions

In [ ]:
def draw_segments(ax, segments, color="red", linewidth=1.5, label=None):
    """Draw line segments on a matplotlib axis."""
    for i, seg in enumerate(segments):
        x1, y1 = seg.start_point().x(), seg.start_point().y()
        x2, y2 = seg.end_point().x(), seg.end_point().y()
        ax.plot([x1, x2], [y1, y2], color=color, linewidth=linewidth,
                label=label if i == 0 else None)


def show_segments(img, segment_lists, titles, figsize=(14, 5)):
    """Show an image with overlaid segment sets side by side."""
    n = len(segment_lists)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    colors = ["red", "lime", "cyan", "magenta", "yellow"]
    for ax, segs, title in zip(axes, segment_lists, titles):
        ax.imshow(img, cmap="gray")
        draw_segments(ax, segs, color=colors[0])
        ax.set_title(f"{title} ({len(segs)} segs)")
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def make_test_image(size=200):
    """Create a simple synthetic test image with known line features."""
    img = np.zeros((size, size), dtype=np.uint8)
    # Horizontal line
    img[50, 20:180] = 255
    # Vertical line
    img[30:170, 100] = 255
    # Diagonal line
    for i in range(140):
        r, c = 30 + i, 30 + i
        if 0 <= r < size and 0 <= c < size:
            img[r, c] = 255
    # Add some contrast variation
    img = np.clip(img.astype(np.int16) + 30, 0, 255).astype(np.uint8)
    return img


# Create test image and segments for use throughout the tutorial
test_img = make_test_image()
plt.imshow(test_img, cmap="gray")
plt.title("Test Image")
plt.axis("off")
plt.show()

---
## 2. LineMerge — Merging Collinear Segments

After detection, line segments are often fragmented. `LineMerge` iteratively
merges pairs that are approximately collinear, based on four criteria:

| Parameter | Default | Description |
|-----------|---------|-------------|
| `max_dist` | 20.0 | Maximum endpoint distance (px) |
| `angle_error` | 5.0 | Maximum angle difference (degrees) |
| `distance_error` | 3.0 | Maximum perpendicular distance (px) |
| `parallel_error` | 10.0 | Maximum parallel gap (px) |
| `merge_type` | `STANDARD` | `STANDARD` (furthest endpoints) or `AVG` |

In [ ]:
# Two nearly collinear horizontal fragments
seg_a = geo.LineSegment_f64.from_endpoints(10, 50, 80, 50)
seg_b = geo.LineSegment_f64.from_endpoints(85, 50, 180, 50)

# A perpendicular segment that should NOT be merged
seg_c = geo.LineSegment_f64.from_endpoints(100, 20, 100, 150)

input_segs = [seg_a, seg_b, seg_c]
print(f"Input: {len(input_segs)} segments")

# Merge with default parameters
merger = alg.LineMerge(max_dist=20.0, angle_error=10.0)
merged = merger.merge_lines(input_segs)
print(f"After merge: {len(merged)} segments")

# Visualize
show_segments(test_img, [input_segs, merged], ["Before Merge", "After Merge"])

In [ ]:
# Compare STANDARD vs AVG merge types
seg1 = geo.LineSegment_f64.from_endpoints(10, 30, 50, 32)
seg2 = geo.LineSegment_f64.from_endpoints(55, 28, 100, 30)

merger_std = alg.LineMerge(max_dist=20, angle_error=10,
                           merge_type=alg.MergeType.STANDARD)
merger_avg = alg.LineMerge(max_dist=20, angle_error=10,
                           merge_type=alg.MergeType.AVG)

merged_std = merger_std.merge_lines([seg1, seg2])
merged_avg = merger_avg.merge_lines([seg1, seg2])

print(f"STANDARD: {len(merged_std)} segment(s)")
if merged_std:
    s = merged_std[0]
    print(f"  ({s.start_point().x():.1f}, {s.start_point().y():.1f}) -> "
          f"({s.end_point().x():.1f}, {s.end_point().y():.1f})")

print(f"AVG: {len(merged_avg)} segment(s)")
if merged_avg:
    s = merged_avg[0]
    print(f"  ({s.start_point().x():.1f}, {s.start_point().y():.1f}) -> "
          f"({s.end_point().x():.1f}, {s.end_point().y():.1f})")

### Exercise 4.1

Create **5 short collinear horizontal fragments** spaced 8 pixels apart
(e.g., at `y=100`, from `x=0..20`, `x=28..48`, etc.).
Merge them with `max_dist=15`, `angle_error=5`. How many segments remain?
What happens if you reduce `max_dist` to 5?

In [ ]:
# TODO: Create 5 fragments, merge with max_dist=15, then with max_dist=5.
#       Print the number of segments after each merge.

**Solution**

In [ ]:
fragments = [
    geo.LineSegment_f64.from_endpoints(0 + i * 28, 100, 20 + i * 28, 100)
    for i in range(5)
]
print(f"Input: {len(fragments)} fragments")

# max_dist=15 — gap is 8 px → should merge all
m1 = alg.LineMerge(max_dist=15, angle_error=5).merge_lines(fragments)
print(f"max_dist=15: {len(m1)} segment(s)")

# max_dist=5 — gap is 8 px → too far, no merges
m2 = alg.LineMerge(max_dist=5, angle_error=5).merge_lines(fragments)
print(f"max_dist=5:  {len(m2)} segment(s)")

With `max_dist=15` the 8-pixel gaps are within range and all 5 fragments
merge into a single segment. With `max_dist=5` no fragment pair is close
enough, so all 5 remain separate.

---
## 3. LineConnect — Gradient-Guided Connection

`LineConnect` joins nearby segment endpoints when the gradient magnitude
along the connecting path is strong enough. Unlike `LineMerge`, it uses
the image gradient to verify connections.

| Parameter | Default | Description |
|-----------|---------|-------------|
| `max_radius` | 15.0 | Maximum endpoint distance to consider (px) |
| `accuracy` | 2.0 | Sampling step along the connecting path (px) |
| `threshold` | 10.0 | Minimum average gradient magnitude |

In [ ]:
# LineConnect requires a gradient magnitude image
connector = alg.LineConnect(max_radius=20.0, accuracy=2.0, threshold=5.0)

# Create a simple gradient magnitude map (using the test image itself)
magnitude = test_img.astype(np.float64)

# Two segments near each other
segs = [
    geo.LineSegment_f64.from_endpoints(10, 50, 70, 50),
    geo.LineSegment_f64.from_endpoints(80, 50, 180, 50),
]
print(f"Before connect: {len(segs)} segments")

connected = connector.connect_lines(segs, magnitude)
print(f"After connect:  {len(connected)} segments")

> **Note:** `LineConnect` evaluates all four endpoint pairings
> (start–start, start–end, end–start, end–end) for each candidate pair
> and picks the one with the highest gradient response.

---
## 4. AccuracyMeasure — Evaluating Detection Quality

`AccuracyMeasure` computes standard detection metrics by matching detected
segments to ground truth. A detected segment matches when the minimum of
the two endpoint distance averages (forward and reversed) is below the
threshold.

### Metrics

| Metric | Formula | Description |
|--------|---------|-------------|
| Precision | TP / (TP + FP) | Fraction of detections that are correct |
| Recall | TP / (TP + FN) | Fraction of GT segments that are detected |
| F1 | 2·P·R / (P+R) | Harmonic mean of precision and recall |
| sAP | avg(F1 across thresholds) | Structural Average Precision |

In [ ]:
# Ground truth
gt = [
    geo.LineSegment_f64.from_endpoints(10, 50, 180, 50),   # horizontal
    geo.LineSegment_f64.from_endpoints(100, 30, 100, 170),  # vertical
]

# Perfect detection
detected_perfect = [
    geo.LineSegment_f64.from_endpoints(10, 50, 180, 50),
    geo.LineSegment_f64.from_endpoints(100, 30, 100, 170),
]

measure = alg.AccuracyMeasure(threshold=5.0)
result = measure.evaluate(detected_perfect, gt)
print("=== Perfect Detection ===")
print(f"Precision: {result.precision:.3f}")
print(f"Recall:    {result.recall:.3f}")
print(f"F1:        {result.f1:.3f}")
print(f"TP={result.true_positives}  FP={result.false_positives}  FN={result.false_negatives}")

In [ ]:
# Imperfect detection: 1 correct, 1 missed, 1 false alarm
detected_partial = [
    geo.LineSegment_f64.from_endpoints(10, 50, 180, 50),   # matches GT[0]
    geo.LineSegment_f64.from_endpoints(50, 80, 150, 80),   # false positive
]

result2 = measure.evaluate(detected_partial, gt)
print("=== Partial Detection ===")
print(f"Precision: {result2.precision:.3f}")
print(f"Recall:    {result2.recall:.3f}")
print(f"F1:        {result2.f1:.3f}")
print(f"TP={result2.true_positives}  FP={result2.false_positives}  FN={result2.false_negatives}")

In [ ]:
# Structural AP: averages F1 across multiple distance thresholds
sap = measure.structural_ap(detected_perfect, gt, thresholds=[5, 10, 15])
print(f"sAP (perfect):  {sap:.3f}")

sap2 = measure.structural_ap(detected_partial, gt, thresholds=[5, 10, 15])
print(f"sAP (partial):  {sap2:.3f}")

### Exercise 4.2

Create a ground truth with 4 line segments (e.g., a rectangle).
Then create a detected set that:
- Matches 2 of the 4 GT segments
- Has 1 extra false positive

Compute precision, recall, and F1. Verify manually that your computed
values match the expected formulas.

In [ ]:
# TODO: Define ground truth (4 segments), detected (3 segments with 2 TP, 1 FP).
#       Evaluate and verify against P=TP/(TP+FP), R=TP/(TP+FN).

**Solution**

In [ ]:
# Ground truth: rectangle
gt_rect = [
    geo.LineSegment_f64.from_endpoints(10, 10, 90, 10),   # top
    geo.LineSegment_f64.from_endpoints(90, 10, 90, 90),   # right
    geo.LineSegment_f64.from_endpoints(90, 90, 10, 90),   # bottom
    geo.LineSegment_f64.from_endpoints(10, 90, 10, 10),   # left
]

# Detected: match top + right, plus one false alarm
det_rect = [
    geo.LineSegment_f64.from_endpoints(10, 10, 90, 10),   # matches top
    geo.LineSegment_f64.from_endpoints(90, 10, 90, 90),   # matches right
    geo.LineSegment_f64.from_endpoints(50, 50, 80, 70),   # false positive
]

measure = alg.AccuracyMeasure(threshold=5.0)
result = measure.evaluate(det_rect, gt_rect)

print(f"TP={result.true_positives}, FP={result.false_positives}, FN={result.false_negatives}")
print(f"Precision: {result.precision:.4f}  (expected: 2/3 = {2/3:.4f})")
print(f"Recall:    {result.recall:.4f}  (expected: 2/4 = {2/4:.4f})")
expected_f1 = 2 * (2/3) * (2/4) / ((2/3) + (2/4))
print(f"F1:        {result.f1:.4f}  (expected: {expected_f1:.4f})")

With 2 TP, 1 FP, 2 FN: Precision = 2/3 ≈ 0.667, Recall = 2/4 = 0.5,
F1 = 2·(2/3)·(1/2) / (2/3 + 1/2) ≈ 0.571.

---
## 5. GroundTruthLoader — CSV I/O

`GroundTruthLoader` reads and writes ground truth annotations in CSV format.
Segments are grouped by image name.

```
image_name,x1,y1,x2,y2
img001.png,10,20,100,20
img001.png,50,10,50,90
```

In [ ]:
# Create ground truth entries programmatically
segments_a = [
    geo.LineSegment_f64.from_endpoints(10, 10, 90, 10),
    geo.LineSegment_f64.from_endpoints(50, 10, 50, 90),
]
entry_a = alg.GroundTruthLoader.make_entry("image_a.png", segments_a)

segments_b = [
    geo.LineSegment_f64.from_endpoints(0, 0, 100, 100),
]
entry_b = alg.GroundTruthLoader.make_entry("image_b.png", segments_b)

print(entry_a)  # <GroundTruthEntry('image_a.png', 2 segments)>
print(entry_b)  # <GroundTruthEntry('image_b.png', 1 segments)>
print(f"Entry A has {len(entry_a.segments)} segments for '{entry_a.image_name}'")

In [ ]:
# Save and reload (to a temp file)
import tempfile, os

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False, mode="w") as f:
    tmp_path = f.name

alg.GroundTruthLoader.save_csv(tmp_path, [entry_a, entry_b])

# Read back and verify
loaded = alg.GroundTruthLoader.load_csv(tmp_path)
for entry in loaded:
    print(f"{entry.image_name}: {len(entry.segments)} segments")

# Print raw CSV content
print("\nCSV content:")
with open(tmp_path) as f:
    print(f.read())

os.unlink(tmp_path)

---
## 6. ImageAnalyzer — Image Property Analysis

`ImageAnalyzer` extracts measurable properties from an image, all
normalized to [0, 1]:

| Property | Derived From | Description |
|----------|-------------|-------------|
| `contrast` | `meanStdDev` | Normalized intensity std-dev |
| `noise_level` | MAD of Laplacian | Robust noise estimate |
| `edge_density` | Sobel threshold | Fraction of strong-gradient pixels |
| `dynamic_range` | Histogram 5th–95th percentile | Spread of intensity values |

These properties drive the `suggest_profile()` heuristic that produces
adaptive knob values for `DetectorProfile`.

In [ ]:
# Analyze the synthetic test image
props = alg.ImageAnalyzer.analyze(test_img)
print(f"Test image properties:")
print(f"  Contrast:      {props.contrast:.4f}")
print(f"  Noise level:   {props.noise_level:.4f}")
print(f"  Edge density:  {props.edge_density:.4f}")
print(f"  Dynamic range: {props.dynamic_range:.4f}")
print(f"\nRepr: {props}")

In [ ]:
# Compare different image types
imgs = {
    "Uniform (flat)": np.full((200, 200), 128, dtype=np.uint8),
    "High contrast": np.vstack([
        np.zeros((100, 200), dtype=np.uint8),
        np.full((100, 200), 255, dtype=np.uint8),
    ]),
    "Noisy": np.random.randint(0, 255, (200, 200), dtype=np.uint8),
    "Gradient": np.tile(
        np.linspace(0, 255, 200).astype(np.uint8), (200, 1)
    ),
}

fig, axes = plt.subplots(1, len(imgs), figsize=(16, 4))
for ax, (name, img) in zip(axes, imgs.items()):
    props = alg.ImageAnalyzer.analyze(img)
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(f"{name}\nC={props.contrast:.2f} N={props.noise_level:.2f}\n"
                 f"E={props.edge_density:.2f} R={props.dynamic_range:.2f}",
                 fontsize=9)
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# Profile suggestions from image analysis
for name, img in imgs.items():
    props = alg.ImageAnalyzer.analyze(img)
    hints = props.suggest_profile()
    print(f"{name:20s} → detail={hints.detail:5.1f}%  gap_tol={hints.gap_tolerance:5.1f}%  "
          f"min_len={hints.min_length:5.1f}%  prec={hints.precision:5.1f}%  "
          f"c_factor={hints.contrast_factor:.2f}  n_factor={hints.noise_factor:.2f}")

### Exercise 4.3

Create three 300×300 images:
1. A **low noise** image: draw a white cross on a dark gray background (value 40)
2. The **same image** with Gaussian noise added (`np.random.normal`, sigma=30)
3. A **high edge density** image: a 10×10 checkerboard pattern

Analyze all three and compare the `noise_level` and `edge_density` values.
Which image gets the highest `min_length` suggestion? Why?

In [ ]:
# TODO: Create the three images, analyze them, print properties and hints.

**Solution**

In [ ]:
# 1. Low noise cross
clean = np.full((300, 300), 40, dtype=np.uint8)
clean[140:160, 50:250] = 220  # horizontal bar
clean[50:250, 140:160] = 220  # vertical bar

# 2. Same with noise
noisy = np.clip(
    clean.astype(np.float64) + np.random.normal(0, 30, clean.shape),
    0, 255
).astype(np.uint8)

# 3. Checkerboard
checker = np.zeros((300, 300), dtype=np.uint8)
for r in range(10):
    for c in range(10):
        if (r + c) % 2 == 0:
            checker[r*30:(r+1)*30, c*30:(c+1)*30] = 255

test_imgs = {"Clean cross": clean, "Noisy cross": noisy, "Checkerboard": checker}

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, (name, img) in zip(axes, test_imgs.items()):
    props = alg.ImageAnalyzer.analyze(img)
    hints = props.suggest_profile()
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(f"{name}\nnoise={props.noise_level:.3f}, edges={props.edge_density:.3f}\n"
                 f"min_len hint={hints.min_length:.0f}%", fontsize=9)
    ax.axis("off")
    print(f"{name:16s}: noise={props.noise_level:.3f}, edges={props.edge_density:.3f}, "
          f"min_len_hint={hints.min_length:.1f}%")
plt.tight_layout()
plt.show()

The noisy or checkerboard image typically gets the highest `min_length`
suggestion, because the heuristic increases `min_length` with both noise
and edge density to filter out spurious short detections.

---
## 7. DetectorProfile — Intuitive Parameter Knobs

`DetectorProfile` translates 4 human-readable percentage knobs into
concrete parameters for any of the 9 supported LSD detectors.

| Knob | Low (0%) | High (100%) |
|------|----------|-------------|
| **detail** | Coarse / salient edges only | Fine details included |
| **gap_tolerance** | No gaps allowed | Very tolerant of gaps |
| **min_length** | Keep all (even tiny) | Long segments only |
| **precision** | Rough / fast | Sub-pixel accurate |

### Supported Detectors

| DetectorId | Name |
|-----------|------|
| `LSD_CC` | LsdCC |
| `LSD_CP` | LsdCP |
| `LSD_BURNS` | LsdBurns |
| `LSD_FBW` | LsdFBW |
| `LSD_FGIOI` | LsdFGioi |
| `LSD_EDLZ` | LsdEDLZ |
| `LSD_EL` | LsdEL |
| `LSD_EP` | LsdEP |
| `LSD_HOUGHP` | LsdHoughP |

In [ ]:
# Creating a profile with manual knob values
profile = alg.DetectorProfile(detail=70, gap_tolerance=30,
                              min_length=50, precision=80)
print(profile)

# List all supported detectors
print(f"\nSupported detectors: {alg.DetectorProfile.supported_detectors()}")

In [ ]:
# Generate concrete parameters for LsdFGioi
params = profile.to_params(alg.DetectorId.LSD_FGIOI)
print("LsdFGioi parameters:")
for p in params:
    print(f"  {p['name']:20s} = {p['value']}")

# Same by name string
params2 = profile.to_params_by_name("LsdFGioi")
assert params == params2, "Both methods should give identical results"

In [ ]:
# Compare parameters across all detectors
profile = alg.DetectorProfile(detail=50, gap_tolerance=50,
                              min_length=50, precision=50)

for name in alg.DetectorProfile.supported_detectors():
    params = profile.to_params_by_name(name)
    param_str = ", ".join(f"{p['name']}={p['value']}" for p in params[:3])
    if len(params) > 3:
        param_str += f", ... (+{len(params)-3} more)"
    print(f"{name:12s}: {param_str}")

In [ ]:
# Effect of the detail knob on threshold parameters
detail_values = [0, 25, 50, 75, 100]
detector_name = "LsdFGioi"

print(f"Detail knob effect on {detector_name}:")
print(f"{'Detail':>8s}  {'quant_error':>12s}  {'angle_th':>10s}")
print("-" * 35)

for d in detail_values:
    p = alg.DetectorProfile(detail=d)
    params = {x['name']: x['value'] for x in p.to_params_by_name(detector_name)}
    print(f"{d:>7d}%  {params.get('quant_error', '-'):>12}  "
          f"{params.get('angle_th', '-'):>10}")

### 7.1 Image-Adaptive Profiles

The most powerful workflow: analyze an image, get adaptive knob suggestions,
and then generate detector parameters — all in a pipeline.

In [ ]:
# Image-adaptive workflow
img = test_img

# Step 1: Analyze
props = alg.ImageAnalyzer.analyze(img)
print(f"Image: contrast={props.contrast:.3f}, noise={props.noise_level:.3f}")

# Step 2: Get hints
hints = props.suggest_profile()
print(f"Hints: detail={hints.detail:.0f}%, gap={hints.gap_tolerance:.0f}%, "
      f"len={hints.min_length:.0f}%, prec={hints.precision:.0f}%")
print(f"Factors: contrast={hints.contrast_factor:.2f}, noise={hints.noise_factor:.2f}")

# Step 3: Create profile from hints (carries adaptive factors)
profile = alg.DetectorProfile.from_hints(hints)

# Or the shortcut:
profile_auto = alg.DetectorProfile.from_image(img)

# Step 4: Generate detector parameters
for name in ["LsdCC", "LsdFGioi", "LsdHoughP"]:
    params = profile.to_params_by_name(name)
    print(f"\n{name}: {len(params)} params")
    for p in params:
        print(f"  {p['name']} = {p['value']}")

In [ ]:
# Modifying knobs after creation
profile = alg.DetectorProfile.from_image(test_img)
print(f"Auto:     detail={profile.detail:.0f}%, precision={profile.precision:.0f}%")

# Override: user wants maximum detail
profile.detail = 95
# And override the adaptive noise factor
profile.noise_factor = 1.0
print(f"Override: detail={profile.detail:.0f}%, precision={profile.precision:.0f}%")
print(f"Factors:  contrast={profile.contrast_factor:.2f}, noise={profile.noise_factor:.2f}")

### Exercise 4.4

Write a function `compare_profiles(img, detail_values)` that:

1. Creates a `DetectorProfile.from_image(img)` as a baseline.
2. For each value in `detail_values` (e.g., `[10, 50, 90]`), creates a profile
   copy with that `detail` setting but keeps all other knobs and factors from
   the baseline.
3. Prints the LsdFGioi parameters for each detail level.

Use it on the test image. Which parameter changes the most?

In [ ]:
# TODO: Implement compare_profiles and call it.

**Solution**

In [ ]:
def compare_profiles(img, detail_values):
    """Compare LsdFGioi parameters across different detail settings."""
    baseline = alg.DetectorProfile.from_image(img)
    print(f"Baseline: detail={baseline.detail:.0f}%, gap={baseline.gap_tolerance:.0f}%, "
          f"len={baseline.min_length:.0f}%, prec={baseline.precision:.0f}%")
    print(f"Factors:  contrast={baseline.contrast_factor:.2f}, noise={baseline.noise_factor:.2f}")
    print()

    all_params = {}
    for d in detail_values:
        profile = alg.DetectorProfile(
            detail=d,
            gap_tolerance=baseline.gap_tolerance,
            min_length=baseline.min_length,
            precision=baseline.precision,
        )
        profile.contrast_factor = baseline.contrast_factor
        profile.noise_factor = baseline.noise_factor

        params = {p['name']: p['value'] for p in profile.to_params_by_name("LsdFGioi")}
        all_params[d] = params

    # Print comparison table
    param_names = list(all_params[detail_values[0]].keys())
    header = f"{'Param':>15s}" + "".join(f"  d={d:3d}%" for d in detail_values)
    print(header)
    print("-" * len(header))
    for pname in param_names:
        vals = [f"  {all_params[d][pname]:>6}" for d in detail_values]
        print(f"{pname:>15s}" + "".join(vals))

compare_profiles(test_img, [10, 30, 50, 70, 90])

The `quant_error` parameter typically shows the largest relative change
(decreasing from ~3.0 to ~0.5), since it directly controls how sensitive
the LsdFGioi detector is to deviations from perfect straightness.

In [ ]:
# DetectorId <-> name conversion utilities
did = alg.DetectorId.LSD_FGIOI
name = alg.detector_id_to_name(did)
back = alg.detector_name_to_id(name)

print(f"Enum -> name: {did} -> '{name}'")
print(f"Name -> enum: '{name}' -> {back}")
print(f"Round-trip OK: {did == back}")

# All DetectorId values
for d in alg.DetectorId:
    if d.name != "COUNT":
        print(f"  {d.name:12s} = {d.value}  ->  {alg.detector_id_to_name(d)}")

---
## 8. ParamOptimizer — Automated Hyperparameter Search

`ParamOptimizer` orchestrates a parameter search:

1. Generates candidate configurations via a `SearchStrategy`
2. Runs a user-supplied detection function with each configuration
3. Evaluates output against ground truth using `AccuracyMeasure`
4. Tracks the best result and optionally reports progress

### Search Strategies

| Strategy | Class | Description |
|----------|-------|-------------|
| Grid | `GridSearchStrategy` | Exhaustive Cartesian product |
| Random | `RandomSearchStrategy` | Uniform random sampling |

### ParamRange

Defines a single parameter's search bounds.

```python
alg.ParamRange("alpha", 0.1, 2.0, 0.1)        # float
alg.ParamRange.make_int("k", 3, 15, 2)         # integer
alg.ParamRange.make_bool("refine")              # boolean
```

### OptimMetric

| Value | Optimizes for |
|-------|---------------|
| `F1` | Harmonic mean of precision and recall |
| `PRECISION` | Precision only |
| `RECALL` | Recall only |

In [ ]:
# Define a search space
space = [
    alg.ParamRange("sensitivity", 0.0, 1.0, 0.2),
    alg.ParamRange.make_bool("refine"),
]

# Grid search: generate all combinations
grid = alg.GridSearchStrategy()
configs = grid.generate(space)
print(f"Grid: {len(configs)} configurations")
for i, cfg in enumerate(configs[:4]):
    print(f"  [{i}] {cfg}")
print(f"  ... ({len(configs)} total)")

In [ ]:
# Random search: fixed number of samples
rand = alg.RandomSearchStrategy(num_samples=10, seed=42)
rand_configs = rand.generate(space)
print(f"Random: {len(rand_configs)} configurations")
for cfg in rand_configs[:3]:
    print(f"  {cfg}")

# Reproducible: same seed -> same configs
rand2 = alg.RandomSearchStrategy(num_samples=10, seed=42)
assert rand.generate(space) == rand2.generate(space)
print("\nReproducibility check passed!")

In [ ]:
# Full optimization example

# 1. Prepare ground truth
gt_segments = [
    geo.LineSegment_f64.from_endpoints(10, 50, 180, 50),
    geo.LineSegment_f64.from_endpoints(100, 30, 100, 170),
]
ground_truth = [alg.GroundTruthLoader.make_entry("test.png", gt_segments)]

# 2. Prepare images
images = [("test.png", test_img)]

# 3. Search space
space = [
    alg.ParamRange("sensitivity", 0.0, 1.0, 0.1),
]

# 4. Detection function (simplified: returns GT when sensitivity >= 0.3)
def detect_fn(src, params):
    param_dict = {p["name"]: p["value"] for p in params}
    sensitivity = float(param_dict["sensitivity"])
    if sensitivity >= 0.3:
        return gt_segments  # perfect detection
    return []  # no detections

# 5. Run optimization
optimizer = alg.ParamOptimizer(metric=alg.OptimMetric.F1, match_threshold=5.0)
strategy = alg.GridSearchStrategy()

scores = []
def progress(step, total, best_score):
    scores.append(best_score)
    return True

result = optimizer.optimize(strategy, space, images, ground_truth,
                            detect_fn, progress=progress)

# 6. Results
print(f"Best F1: {result.best_score:.3f}")
print(f"Best params: {result.best_params}")
print(f"Total configs evaluated: {result.total_configs}")

In [ ]:
# Visualize the optimization progress
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(scores) + 1), scores, "o-")
plt.xlabel("Step")
plt.ylabel("Best F1 Score")
plt.title("Optimization Progress")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Inspect top results
top3 = result.top_n(3)
print("Top 3 configurations:")
for i, r in enumerate(top3):
    print(f"  #{i+1}: score={r.score:.3f}  params={r.params}")

### Exercise 4.5

Create a detection function that simulates **noisy** results:

- When `sensitivity < 0.2`: returns empty (no detections)
- When `0.2 <= sensitivity < 0.5`: returns 1 of 3 GT segments
- When `0.5 <= sensitivity < 0.8`: returns all 3 GT segments
- When `sensitivity >= 0.8`: returns all 3 GT segments + 2 false positives

Run a grid search with step 0.05 and compare the best F1 against the best
precision. Which sensitivity achieves each?

In [ ]:
# TODO: Implement the noisy detection function, run grid search for F1 and PRECISION.

**Solution**

In [ ]:
# Ground truth: 3 segments
gt3 = [
    geo.LineSegment_f64.from_endpoints(10, 30, 90, 30),
    geo.LineSegment_f64.from_endpoints(10, 60, 90, 60),
    geo.LineSegment_f64.from_endpoints(10, 90, 90, 90),
]
gt_entries = [alg.GroundTruthLoader.make_entry("test.png", gt3)]
images = [("test.png", test_img)]

# False positives
fp1 = geo.LineSegment_f64.from_endpoints(20, 45, 80, 45)
fp2 = geo.LineSegment_f64.from_endpoints(20, 75, 80, 75)

def noisy_detect(src, params):
    s = float({p["name"]: p["value"] for p in params}["sensitivity"])
    if s < 0.2:
        return []
    elif s < 0.5:
        return [gt3[0]]
    elif s < 0.8:
        return list(gt3)
    else:
        return list(gt3) + [fp1, fp2]

space = [alg.ParamRange("sensitivity", 0.0, 1.0, 0.05)]
strategy = alg.GridSearchStrategy()

# Optimize for F1
opt_f1 = alg.ParamOptimizer(metric=alg.OptimMetric.F1, match_threshold=5.0)
res_f1 = opt_f1.optimize(strategy, space, images, gt_entries, noisy_detect)
print(f"Best F1:        {res_f1.best_score:.3f}  at sensitivity={res_f1.best_params[0]['value']}")

# Optimize for PRECISION
opt_p = alg.ParamOptimizer(metric=alg.OptimMetric.PRECISION, match_threshold=5.0)
res_p = opt_p.optimize(strategy, space, images, gt_entries, noisy_detect)
print(f"Best Precision: {res_p.best_score:.3f}  at sensitivity={res_p.best_params[0]['value']}")

The best F1 is achieved in the 0.5–0.8 range (all GT matched, no FP).
The best precision is also perfect there, but for sensitivity < 0.5 only
1 of 3 GT segments is matched (lower recall). At ≥0.8 the false positives
lower precision from 1.0 to 3/5 = 0.6.

---
## 9. Putting It All Together

This section shows a complete workflow combining ImageAnalyzer,
DetectorProfile, AccuracyMeasure, and LineMerge.

In [ ]:
# Complete end-to-end pipeline

# 1. Analyze the image
img = test_img
props = alg.ImageAnalyzer.analyze(img)
print(f"Image analysis:")
print(f"  {props}")

# 2. Get adaptive profile
profile = alg.DetectorProfile.from_image(img)
print(f"\nProfile: detail={profile.detail:.0f}%, gap={profile.gap_tolerance:.0f}%, "
      f"len={profile.min_length:.0f}%, prec={profile.precision:.0f}%")

# 3. Generate parameters for multiple detectors
for det_name in ["LsdCC", "LsdFGioi", "LsdEDLZ", "LsdHoughP"]:
    params = profile.to_params_by_name(det_name)
    print(f"\n{det_name} ({len(params)} params):")
    for p in params:
        print(f"    {p['name']:20s} = {p['value']}")

# 4. Simulate detection + merge + evaluate
gt_segs = [
    geo.LineSegment_f64.from_endpoints(20, 50, 180, 50),
    geo.LineSegment_f64.from_endpoints(100, 30, 100, 170),
]

# Simulated fragmented detections
detected_fragments = [
    geo.LineSegment_f64.from_endpoints(20, 50, 90, 50),   # left half
    geo.LineSegment_f64.from_endpoints(100, 50, 180, 50), # right half
    geo.LineSegment_f64.from_endpoints(100, 30, 100, 170), # vertical
]

# Before merge
measure = alg.AccuracyMeasure(threshold=10.0)
r_before = measure.evaluate(detected_fragments, gt_segs)
print(f"\nBefore merge: P={r_before.precision:.3f}  R={r_before.recall:.3f}  F1={r_before.f1:.3f}")

# Merge
merger = alg.LineMerge(max_dist=15, angle_error=10)
merged = merger.merge_lines(detected_fragments)
print(f"Merged: {len(detected_fragments)} -> {len(merged)} segments")

# After merge
r_after = measure.evaluate(merged, gt_segs)
print(f"After merge:  P={r_after.precision:.3f}  R={r_after.recall:.3f}  F1={r_after.f1:.3f}")

---
## 10. Comprehension Questions

Test your understanding of the algorithm library concepts.

---

**Q1:** What is the difference between `LineMerge` and `LineConnect`?
When would you use one over the other?

<details>
<summary>Answer</summary>

`LineMerge` combines segments based on geometric criteria only (distance,
angle, parallelism) — no image data is needed. `LineConnect` uses the
gradient magnitude along the connecting path, requiring an image.

Use `LineMerge` after detection to consolidate collinear fragments.
Use `LineConnect` when you want to bridge gaps only where there is
evidence of an edge in the image.

</details>

---

**Q2:** If a detector produces 5 segments, of which 3 match the
ground truth (which has 4 segments), what are the precision, recall,
and F1 values?

<details>
<summary>Answer</summary>

- TP = 3, FP = 2 (5 detected − 3 matched), FN = 1 (4 GT − 3 matched)
- Precision = 3/5 = 0.600
- Recall = 3/4 = 0.750
- F1 = 2 · 0.6 · 0.75 / (0.6 + 0.75) = 0.667

</details>

---

**Q3:** What does the `contrast_factor` in `ProfileHints` do, and how
does it affect detection?

<details>
<summary>Answer</summary>

The `contrast_factor` is a multiplicative scaling factor (range 0.5–2.0)
applied to threshold-like parameters during `to_params()`. For a
low-contrast image, `contrast_factor > 1.0`, which raises detection
thresholds so that the detector does not generate excessive spurious
detections. For a high-contrast image, `contrast_factor < 1.0`, allowing
the detector to be more sensitive.

</details>

---

**Q4:** Why might the `min_length` suggestion increase for noisy images?

<details>
<summary>Answer</summary>

In noisy images, short edge chains are more likely to be noise artifacts
rather than real structure. By increasing `min_length`, the profile tells
the detector to discard very short segments, effectively filtering noise
at the cost of missing genuinely short line features.

</details>

---

**Q5:** What is the difference between `GridSearchStrategy` and
`RandomSearchStrategy`? In what scenario is random search preferable?

<details>
<summary>Answer</summary>

`GridSearchStrategy` evaluates the exhaustive Cartesian product of all
parameter values — guaranteed to find the optimum within the grid, but
grows exponentially with dimensions. `RandomSearchStrategy` samples
uniformly at random, which scales better in high dimensions and is
preferable when the search space is large (many parameters or fine step
sizes) since it can explore the space more efficiently.

</details>

---

**Q6:** Explain the end-to-end image-adaptive detection workflow using
`ImageAnalyzer` and `DetectorProfile`. What are the steps?

<details>
<summary>Answer</summary>

1. **Analyze** the image with `ImageAnalyzer.analyze(img)` to get
   `ImageProperties`.
2. Call `suggest_profile()` on the properties to get `ProfileHints`
   (knob suggestions + adaptive factors).
3. Create a `DetectorProfile` from the hints (`from_hints()` or
   the shortcut `from_image()`).
4. Optionally override specific knobs (e.g., `profile.detail = 80`).
5. Call `to_params()` or `apply()` for the desired detector to get
   concrete parameters adapted to the image characteristics.

The adaptive factors ensure that the same knob settings produce
appropriate detector behavior regardless of image contrast and noise.

</details>

---

**Q7:** Can you use `DetectorProfile` without `ImageAnalyzer`? How?

<details>
<summary>Answer</summary>

Yes. You can construct a `DetectorProfile` with explicit knob values:
```python
profile = alg.DetectorProfile(detail=70, gap_tolerance=30,
                              min_length=50, precision=80)
```
This uses the default adaptive factors (both 1.0). The profile will
generate reasonable parameters without any image analysis. The
`ImageAnalyzer` integration is optional and only needed for automatic
adaptation to image characteristics.

</details>

---

**Q8:** What does `structural_ap` (sAP) measure, and how does it differ
from a single F1 evaluation?

<details>
<summary>Answer</summary>

Structural AP averages the F1 score across **multiple distance
thresholds** (e.g., [5, 10, 15] pixels). A single F1 evaluation only
tells you how well the detector performs at one specific threshold;
sAP provides a more robust measure by rewarding detectors that are
accurate at multiple tolerance levels. A high sAP means the detector
produces well-localized segments across a range of matching criteria.

</details>

---
## 11. Challenge Exercise

### Exercise 4.6

Build a **profile comparison dashboard**:

1. Create 3 different synthetic images (e.g., clean lines, noisy lines,
   dense grid lines).
2. For each image, run `ImageAnalyzer.analyze()` and create a profile
   with `DetectorProfile.from_image()`.
3. For each profile, generate parameters for at least 3 different
   detectors.
4. Create a bar chart or table showing how one specific parameter
   (e.g., `nms_th_low` for LsdCC) varies across the 3 images.

This exercise combines ImageAnalyzer, DetectorProfile, and visualization.

In [ ]:
# TODO: Build the profile comparison dashboard.

**Solution**

In [ ]:
# 1. Create 3 images
clean = np.zeros((200, 200), dtype=np.uint8)
clean[50, 20:180] = 200
clean[100, 20:180] = 200
clean[150, 20:180] = 200

noisy = np.clip(
    clean.astype(np.float64) + np.random.normal(0, 40, clean.shape),
    0, 255
).astype(np.uint8)

dense = np.zeros((200, 200), dtype=np.uint8)
for i in range(0, 200, 10):
    dense[i, :] = 200
    dense[:, i] = 200

test_images = {"Clean lines": clean, "Noisy lines": noisy, "Dense grid": dense}

# 2. Analyze and create profiles
profiles = {}
for name, img in test_images.items():
    profile = alg.DetectorProfile.from_image(img)
    profiles[name] = profile
    props = alg.ImageAnalyzer.analyze(img)
    print(f"{name:15s}: C={props.contrast:.3f} N={props.noise_level:.3f} "
          f"E={props.edge_density:.3f} -> detail={profile.detail:.0f}%")

# 3. Generate params for 3 detectors
detectors = ["LsdCC", "LsdFGioi", "LsdHoughP"]

# 4. Create comparison chart
fig, ax = plt.subplots(figsize=(8, 5))
x = np.arange(len(test_images))
width = 0.25

for i, det in enumerate(detectors):
    values = []
    for img_name in test_images:
        params = {p['name']: p['value'] for p in profiles[img_name].to_params_by_name(det)}
        # Pick first numeric parameter for comparison
        first_param = list(params.values())[0]
        values.append(float(first_param))
    first_key = list({p['name']: p['value'] for p in profiles[list(test_images.keys())[0]].to_params_by_name(det)}.keys())[0]
    ax.bar(x + i * width, values, width, label=f"{det} ({first_key})")

ax.set_xticks(x + width)
ax.set_xticklabels(test_images.keys())
ax.set_ylabel("Parameter Value")
ax.set_title("First Parameter Comparison Across Images")
ax.legend()
plt.tight_layout()
plt.show()

The chart shows how the same detector's parameters change based on
image characteristics. Noisy images typically result in higher thresholds
(due to `noise_factor > 1`), while clean images allow more sensitive
settings.

---
## Summary

| Component | Purpose | Key Methods |
|-----------|---------|-------------|
| `LineMerge` | Consolidate fragmented segments | `merge_lines()` |
| `LineConnect` | Bridge gaps with gradient evidence | `connect_lines()` |
| `AccuracyMeasure` | Evaluate detection quality | `evaluate()`, `structural_ap()` |
| `GroundTruthLoader` | CSV ground truth I/O | `load_csv()`, `save_csv()`, `make_entry()` |
| `ImageAnalyzer` | Extract image properties | `analyze()` -> `ImageProperties` |
| `DetectorProfile` | Intuitive 4-knob -> params | `to_params()`, `from_image()` |
| `ParamOptimizer` | Automated search | `optimize()` -> `SearchResult` |

### Next Steps

- **Tutorial 1**: Review library fundamentals
- **Tutorial 2**: Explore edge & line detection pipelines
- **Tutorial 3**: Dive into evaluation framework
- Try `DetectorProfile` with real images and actual LSD detectors
- Use `ParamOptimizer` with real detection functions for your use case